# SemanticChunker

텍스트를 의미론적 유사성에 기반하여 분할합니다.

**Reference**

- [Greg Kamradt의 노트북](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

이 방법은 텍스트를 문장 단위로 분할한 후, 3개의 문장씩 그룹화하고, 임베딩 공간에서 유사한 문장들을 병합하는 과정을 거칩니다.

> 의미적으로 유사한 내용을 모아주자.
> 
> 어떤 청크는 길수도 있고, 어떤 것은 짧을 수도 있다.

샘플 텍스트를 로드하고 내용을 출력합니다.


In [1]:
# data/appendix-keywords.txt 파일을 열어서 f라는 파일 객체를 생성합니다.
with open("./data/2025_사우회선출.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

# 파일으로부터 읽은 내용을 일부 출력합니다.
print(file[:350])

안녕하세요, 2024년 사우회입니다.
어느덧 2024년이 마무리되어 가는 이 시점에 2025년 사우회를 이끌어주실 분에 대한 투표를 진행하겠습니다.
모두 한 분 한 분의 소중한 한 표를 꼭 행사해 주시기 바랍니다!

투표 방식: 무기명 온라인 1인 1투표
투표 기간: 2024년 12월 23일(월)까지 (1주일간 진행)
투표 링크: https://forms.gle/ooDQvUP4SzJRaUoS6

[노사 협의회 구성원 및 선출 공고 사항]
노사 협의회 구성원 중 근로자 대표는 노사 협의회장을 겸하며, 노사 간 중요 의사결정 과정에서 합의의 주체가 될 수 있습니다.
다만, 제반 제도와 관련된 안내 및 직원 커뮤니케이션 등의


## SemanticChunker 생성

`SemanticChunker`는 LangChain의 실험적 기능 중 하나로, 텍스트를 의미론적으로 유사한 청크로 분할하는 역할을 합니다.

이를 통해 텍스트 데이터를 보다 효과적으로 처리하고 분석할 수 있습니다.


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

`SemanticChunker`를 사용하여 텍스트를 의미적으로 관련된 청크로 분할합니다.


In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), 
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70
)

## 텍스트 분할


- `text_splitter`를 사용하여 `file` 텍스트를 문서 단위로 분할합니다.


In [4]:
chunks = text_splitter.split_text(file)

분할된 청크를 확인합니다.


In [5]:
# 분할된 청크 중 첫 번째 청크를 출력합니다.
print(chunks[0])

안녕하세요, 2024년 사우회입니다. 어느덧 2024년이 마무리되어 가는 이 시점에 2025년 사우회를 이끌어주실 분에 대한 투표를 진행하겠습니다.


`create_documents()` 함수를 사용하여 청크를 문서로 변환할 수 있습니다.


In [6]:
# text_splitter를 사용하여 분할합니다.
docs = text_splitter.create_documents([file])

for doc in docs:
    print(f"------ len: {len(doc.page_content)} -----")
    print(f"{doc.page_content}")# 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.

------ len: 82 -----
안녕하세요, 2024년 사우회입니다. 어느덧 2024년이 마무리되어 가는 이 시점에 2025년 사우회를 이끌어주실 분에 대한 투표를 진행하겠습니다.
------ len: 340 -----
모두 한 분 한 분의 소중한 한 표를 꼭 행사해 주시기 바랍니다! 투표 방식: 무기명 온라인 1인 1투표
투표 기간: 2024년 12월 23일(월)까지 (1주일간 진행)
투표 링크: https://forms.gle/ooDQvUP4SzJRaUoS6

[노사 협의회 구성원 및 선출 공고 사항]
노사 협의회 구성원 중 근로자 대표는 노사 협의회장을 겸하며, 노사 간 중요 의사결정 과정에서 합의의 주체가 될 수 있습니다. 다만, 제반 제도와 관련된 안내 및 직원 커뮤니케이션 등의 업무는 여전히 인사팀에서 진행할 예정이오니, 근로자 대표로 선출된 직원분께서는 큰 부담 없이 선출 결과를 받아주시면 감사하겠습니다.
------ len: 148 -----
2025년 근로자(대표) 위원
사우회장 1명, 총무 1명
사용자 위원과 근로자 위원이 원활한 의사소통을 통해 상호 간의 이해와 협조를 증진함으로써 노사 공동의 이익을 구현합니다. 근로자 대표 (회사발전위원회)
근로기준법상 각종 제도에 대해 사측과 합의를 진행합니다.
------ len: 419 -----
기존 사우회 및 회사발전위원회 위원장 선출 방식과 동일하게 근로자 투표 방식으로 진행됩니다. 단, 근로자 대표는 직원 과반수 이상의 동의를 얻어야 하므로, 선출된 후보자는 최다 추천을 받은 1인에 대해 근로자대표선임서를 통해 과반수 이상의 직원 서명 날인을 받아야 합니다. (오프라인 서명)
후보 선출 기준

근로자 대표 후보 (회사발전위원회)
선출 기준: 역임자를 제외한 스펙트라 근속 10~15년 차 사우를 대상으로 선출. 사우회장 후보
선출 기준: 역임자를 제외한 스펙트라 근속 5~9년 차 사우를 대상으로 선출. 주요 업무: 연중행사 및 사우 경조사(인사팀과 협업) 담당. 총무 후보
선출 기준: 역임

![semantic.png](./data/semantic.png)

## Breakpoints

이 chunker는 문장을 "분리"할 시점을 결정하여 작동합니다. 이는 두 문장 간의 임베딩 차이를 살펴봄으로써 이루어집니다.

그 차이가 특정 임계값을 넘으면 문장이 분리됩니다.

- 참고 영상: https://youtu.be/8OJC21T2SL4?si=PzUtNGYJ_KULq3-w&t=2580

### Percentile

기본적인 분리 방식은 백분위수(`Percentile`) 를 기반으로 합니다.

이 방법에서는 문장 간의 모든 차이를 계산한 다음, 지정한 백분위수를 기준으로 분리합니다.


In [ ]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 유형을 백분위수로 설정합니다.
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)

분할된 결과를 확인합니다.


In [ ]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

`docs`의 길이를 출력합니다.


In [ ]:
print(len(docs))  # docs의 길이를 출력합니다.

### Standard Deviation

이 방법에서는 지정한 `breakpoint_threshold_amount` 표준편차보다 큰 차이가 있는 경우 분할됩니다.

- `breakpoint_threshold_type` 매개변수를 "standard_deviation"으로 설정하여 청크 분할 기준을 표준편차 기반으로 지정합니다.


In [ ]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준으로 표준 편차를 사용합니다.
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=1.25,
)

분할된 결과를 확인합니다.


In [ ]:
# text_splitter를 사용하여 분할합니다.
docs = text_splitter.create_documents([file])

In [ ]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

`docs`의 길이를 출력합니다.


In [ ]:
print(len(docs))  # docs의 길이를 출력합니다.

### Interquartile

이 방법에서는 사분위수 범위(interquartile range)를 사용하여 청크를 분할합니다.


- `breakpoint_threshold_type` 매개변수를 "interquartile"로 설정하여 청크 분할 기준을 사분위수 범위로 지정합니다.


In [ ]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 의미론적 청크 분할기를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 임계값 유형을 사분위수 범위로 설정합니다.
    breakpoint_threshold_type="interquartile",
    breakpoint_threshold_amount=0.5,
)

In [ ]:
# text_splitter를 사용하여 분할합니다.
docs = text_splitter.create_documents([file])

# 결과를 출력합니다.
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

`docs`의 길이를 출력합니다.


In [ ]:
print(len(docs))  # docs의 길이를 출력합니다.